In [17]:
# Import necessary libraries
import pandas as pd, shutil, os
import yfinance as yf
import datetime as dt
import numpy as np
from pathlib import Path

In [ ]:
# Get S&P 500 tickers
sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500_tickers = sp500_tickers.Symbol.to_list()
sp500_tickers

In [ ]:
# Clean Tickers of periods
sp500_tickers = [x.replace(".", "-") for x in sp500_tickers]
sp500_tickers

In [ ]:
# Define Start and End dates for hist. data(5 yr)
end_date = dt.datetime.now().date().isoformat()
start_date = dt.datetime.now() - dt.timedelta(weeks=26)
start_date = start_date.date().isoformat()
start_date

In [ ]:
# Removes old Stock_Data folder and replaces it with a new, empty one
shutil.rmtree("Stock_Data")
os.mkdir("Stock_Data")

In [14]:
sp_data = yf.download(sp5_tickers, period="5d", group_by="Ticker")
sp_data

[*********************100%***********************]  5 of 5 completed


ABBV                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2021-08-09  114.580002  115.190002  113.739998  114.059998  114.059998   
2021-08-10  114.199997  115.029999  112.919998  114.820000  114.820000   
2021-08-11  115.160004  115.339996  113.550003  113.720001  113.720001   
2021-08-12  113.959999  115.730003  113.910004  115.639999  115.639999   
2021-08-13  115.820000  116.760002  115.699997  116.480003  116.480003   

                           ABMD                                      ...  \
             Volume        Open        High         Low       Close  ...   
Date                                                                 ...   
2021-08-09  5277900  345.000000  350.000000  342.950012  346.649994  ...   
2021-08-10  5120200  347.899994  348.000000  332.410004  332.700012  ...   
2021-08-11  6837500  330.700012  334.670013  315.720001  319.880005  ...   
2021-08-12  4864700  318.279999  326.179993  318.279999  322.170013  ...   
2021-08-13  4362461  320.899994  324.920013  319.420013  321.649994  ...   

                   MMM                                          ACN  \
                   Low       Close   Adj Close   Volume        Open   
Date                                                                  
2021-08-09  196.779999  197.429993  197.429993  1193300  322.010010   
2021-08-10  197.699997  199.250000  199.250000  1598400  319.970001   
2021-08-11  199.309998  201.570007  201.570007  2217400  320.619995   
2021-08-12  200.360001  201.429993  201.429993  1231800  319.700012   
2021-08-13  200.289993  200.580002  200.580002  1889957  322.190002   

                                                                     
                  High         Low       Close   Adj Close   Volume  
Date                                                                 
2021-08-09  322.260010  319.380005  319.519989  319.519989  1225100  
2021-08-10  322.059998  319.059998  320.179993  320.179993   918900  
2021-08-11  320.859985  317.079987  319.790009  319.790009  1841600  
2021-08-12  321.829987  318.640015  321.760010  321.760010   910900  
2021-08-13  324.040009  321.579987  323.450012  323.450012   958996  

[5 rows x 30 columns]

In [18]:
csv_path = Path("Stock_Data/Stock_Data.csv")

In [16]:
sp_data.to_csv("Stock_Data/Stock_Data.csv")

In [23]:
csv = pd.read_csv(csv_path)
csv

,Unnamed: 0,ABBV,ABBV.1,ABBV.2,ABBV.3,ABBV.4,ABBV.5,ABMD,ABMD.1,ABMD.2,...,MMM.2,MMM.3,MMM.4,MMM.5,ACN,ACN.1,ACN.2,ACN.3,ACN.4,ACN.5
0,NaN,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,...,Low,Close,Adj Close,Volume,Open,High,Low,Close,Adj Close,Volume
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-09,114.58000183105469,115.19000244140625,113.73999786376953,114.05999755859375,114.05999755859375,5277900,345.0,350.0,342.95001220703125,...,196.77999877929688,197.42999267578125,197.42999267578125,1193300,322.010009765625,322.260009765625,319.3800048828125,319.5199890136719,319.5199890136719,1225100
3,2021-08-10,114.19999694824219,115.02999877929688,112.91999816894531,114.81999969482422,114.81999969482422,5120200,347.8999938964844,348.0,332.4100036621094,...,197.6999969482422,199.25,199.25,1598400,319.9700012207031,322.05999755859375,319.05999755859375,320.17999267578125,320.17999267578125,918900
4,2021-08-11,115.16000366210938,115.33999633789062,113.55000305175781,113.72000122070312,113.72000122070312,6837500,330.70001220703125,334.6700134277344,315.7200012207031,...,199.30999755859375,201.57000732421875,201.57000732421875,2217400,320.6199951171875,320.8599853515625,317.0799865722656,319.7900085449219,319.7900085449219,1841600
5,2021-08-12,113.95999908447266,115.7300033569336,113.91000366210938,115.63999938964844,115.63999938964844,4864700,318.2799987792969,326.17999267578125,318.2799987792969,...,200.36000061035156,201.42999267578125,201.42999267578125,1231800,319.70001220703125,321.8299865722656,318.6400146484375,321.760009765625,321.760009765625,910900
6,2021-08-13,115.81999969482422,116.76000213623047,115.69999694824219,116.4800033569336,116.4800033569336,4362461,320.8999938964844,324.9200134277344,319.4200134277344,...,200.2899932861328,200.5800018310547,200.5800018310547,1889957,322.19000244140625,324.0400085449219,321.5799865722656,323.45001220703125,323.45001220703125,958996


In [ ]:
# def calcRSI (ticker):
#     data = yf.download(ticker, period="5y", groupby="Ticker")
#     data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
#     data["Price_Change"] = data["Adj Close"].pct_change()
#     data["Gains"] = data["Price_Change"].apply(lambda x: x if x > 0 else 0)
#     data["Losses"] = data["Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
#     data["Avg_Up"] = data["Gains"].ewm(span=19).mean()
#     data["Avg_Down"] = data["Losses"].ewm(span=19).mean()
#     data = data.dropna()
#     data["RS"] = data["Avg_Up"]/data["Avg_Down"]
#     data["RSI"] = data["RS"].apply(lambda x: 100 - (100/(x+1)))
#     return data

In [ ]:
def calcRSI (ticker):
    data = yf.download(ticker, start=start_date,end=end_date, group_by="Ticker")
    data["Ticker"] = ticker
#     data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
    data["Price_Change"] = data["Adj Close"].pct_change()
#     temp1 = data['Price_Change']
    data['Gains'] = data.loc[:,"Price_Change"].apply(lambda x: x if x > 0 else 0)
#     data["Gains"] = data.iloc[:,7].apply(lambda x: x if x > 0 else 0)
    data["Losses"] = data.loc[:,"Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
#     data["Losses"] = data.iloc[:,7].apply(lambda x: abs(x) if x < 0 else 0)
#     temp2 = data["Gains"]
#     temp3 = data["Losses"]
    data["Avg_Up"] = data.loc[:,"Gains"].ewm(com=13,adjust=False, min_periods=14).mean()
#     temp4 = data["Avg_Up"]
    data["Avg_Down"] = data.loc[:, "Losses"].ewm(com=13,adjust=False, min_periods=14).mean()
#     temp5 = data["Avg_Down"]

#     temp6 = temp4/temp5
    data["RS"] = data.loc[:, "Avg_Up"]/data.loc[:,"Avg_Down"]
    data["RSI"] = data.loc[:,"RS"].apply(lambda x: 100 - (100/(x+1)))
    data = data.dropna()
#     temp7 = data["RSI"]
#     output_df = pd.DataFrame(columns=["Ticker", "RSI"])
#     output_df = output_df.append({"Ticker" : ticker,
#                                  "RSI" : temp7},
#                                 ignore_index=True)
    today = data.iloc[-1]
#     output_df = pd.DataFrame(columns=["Ticker","RSI"])
#     output_df = pd.merge(output_df,today[["Ticker", "RSI"]], on="Ticker", how="left")
    return today

In [ ]:
calcRSI(sp500_tickers[0])

In [ ]:
sp5_tickers= ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']

In [ ]:
# join_data = pd.DataFrame(index="Tickers",columns="RSI")

In [ ]:
def Todays_RSI_Hi():
    stock_tick_list = []
    stock_rsi_list = []
    count = 0
#     join_data = pd.DataFrame()
    for stock in sp500_tickers:
        stockRSI = calcRSI(stock)
        stock_tick_list.append(stockRSI["Ticker"])
        stock_rsi_list.append(stockRSI["RSI"])
        count+=1
        print(count)
    rsi_df_hi = pd.DataFrame(data=(stock_rsi_list), index=stock_tick_list, columns=["RSI"])
    rsi_df_hi.sort_values(by=["RSI"], ascending=False,inplace=True)

    return rsi_df_hi

In [ ]:
fig = Todays_RSI_Hi()

In [ ]:
fig

In [ ]:
def Todays_RSI_Low():
    stock_tick_list = []
    stock_rsi_list = []
    count = 0
#     join_data = pd.DataFrame()
    for stock in sp5_tickers:
        stockRSI = calcRSI(stock)
        stock_tick_list.append(stockRSI["Ticker"])
        stock_rsi_list.append(stockRSI["RSI"])
        count+=1
        print(count)
    rsi_df_low = pd.DataFrame(data=(stock_rsi_list), index=stock_tick_list, columns=["RSI"])
    rsi_df_low = rsi_df_low.sort_values(by=["RSI"],ascending=True)

    return rsi_df_low

In [ ]:
fig2 = Todays_RSI_Low()

In [ ]:
fig2

In [ ]:
aapl_data = yf.download('AAPL', period="1mo")
aapl_data

In [ ]:
aapl_data['Ticker'] = 'AAPL'
aapl_data

In [ ]:
wmt_data = yf.download("WMT", period="1mo")
wmt_data

In [ ]:
combined = pd.merge(aapl_data, wmt_data, how="left", on=['Date'])

In [ ]:
combined

In [ ]:
""